In [1]:
# Import different kinds of libraries
import os
import cv2
import glob
import keras
import numpy as np
import seaborn as sns
import datetime as dt
import tensorflow as tf
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
from keras import Model, layers
from IPython.display import Image
import matplotlib.font_manager as fm
from tensorflow.keras import callbacks
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from keras.models import load_model, model_from_json, Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import DenseNet121, ResNet50, DenseNet201, InceptionV3, Xception, MobileNetV2, VGG19, VGG16
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Concatenate, Dropout,GlobalAveragePooling2D

2024-06-28 00:33:14.742669: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-28 00:33:14.742768: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-28 00:33:14.871664: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
#path for bangla font
font_path = '/kaggle/input/bangla-font/NotoSansBengalit.ttf'


In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#global height width
height = 256
width = 256
def load_data(directory):
    images = []
    labels = []
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        print(label)
        for image_file in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_file)
            # Error handling
            try:
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Error: Unable to read image '{image_path}'")
                    continue
                image = cv2.resize(image, (height, width))  # Resize image
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = image / 255.0
                images.append(image)
                labels.append(label)
            except Exception as e:
                print(f"Error processing image '{image_path}': {e}")
    return np.array(images), np.array(labels)

# Load data
X, y = load_data('/content/drive/MyDrive/FL23D092/Sign Language Images/Bangla_Sign_Final_datasetV2')

# **background remove**

In [ ]:
!pip install removebg
!pip install rembg

In [ ]:
from rembg import new_session, remove
model_name = "u2net"
rembg_session = new_session(model_name)

In [ ]:
import os
def list_files(folder_path):
    file_paths = []
    # Walk through all the directories and subdirectories
    for root, directories, files in os.walk(folder_path):
        for file in files:
            file_paths.append(os.path.join(root, file))
    return file_paths

In [ ]:
img_list = list_files('/content/drive/MyDrive/FL23D092/Sign Language Images/Bangla_Sign_Final_datasetV2')
img_list

In [ ]:
def get_bg_removed(img):
  output = remove(
    img,
    session=rembg_session,
    alpha_matting=True,
    alpha_matting_foreground_threshold=270,
    alpha_matting_background_threshold=20,
    alpha_matting_erode_size=11,
  )
  return output;

In [ ]:
import matplotlib.pyplot as plt

def print_img(img_path, format='jpg'):
  img = plt.imread(img_path, format=format)
  plt.imshow(img)
  plt.axis('off')
  plt.show()

In [ ]:
from PIL import Image

In [ ]:
base_folder = '/content/drive/MyDrive/FL23D092/Sign Language Images/Bangla_Sign_Final_datasetV2/'
sub_folder = 'Bg_remove/'
extension = 'png'

def gen_bg_removed(img_path):

  file_name = os.path.basename(img_path)
  sub_path = img_path.removeprefix(base_folder).removesuffix(file_name)

  output_path = base_folder + sub_folder + sub_path

  if not os.path.exists(output_path):
    os.makedirs(output_path)

  with open(img_path, 'rb') as i:
      with open(output_path + file_name, 'wb') as o:
          input = i.read()
          output = get_bg_removed(input)
          o.write(output)

In [ ]:
for img_path in img_list:
  gen_bg_removed(img_path)

In [ ]:
root_dir = '/content/drive/MyDrive/FL23D092/Sign Language Images/Bangla_Sign_Final_datasetV2/Bg_remove'
# Count the number of images
def count_total(directory):
    num_images = 0
    for f in os.listdir(directory):
        f_path = os.path.join(directory, f)
        # check if the subdirectory or file exists and is a directory
        if os.path.isdir(f_path):
            num_images += count_total(f_path)
        elif os.path.splitext(f)[1].lower() in image_extensions:
            num_images += 1
    return num_images
# function call
num_images = count_total(root_dir)
print(f'Total number of images: {num_images}')

## **Resize, Grayscale, Increase contrast**

In [ ]:
#Gamma < 1: Darkens the image, reducing the contrast
#Gamma = 1: No change, the image remains unchanged.
#Gamma > 1: Lightens the image, increasing the contrast
def gamma_correction(image, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255.0 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)
# specify the desired size for resizing
desired_size = (256, 256)

# specify the root directory
root_dir = '/content/drive/MyDrive/FL23D092/Sign Language Images/Bangla_Sign_Final_datasetV2/Bg_remove'

# specify the directory to save processed images
save_dir = '/content/drive/MyDrive/FL23D092/Sign Language Images/Bangla_Sign_Final_datasetV2/augmented_data/'

# list of image extensions
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# iterate over all subdirectories in the root directory
for subdir in os.listdir(root_dir):
    sub_dir_path = os.path.join(root_dir, subdir)
    if os.path.isdir(sub_dir_path):
        for filename in os.listdir(sub_dir_path):
            file_path = os.path.join(sub_dir_path, filename)
            if os.path.splitext(file_path)[1].lower() in image_extensions:
                img = cv2.imread(file_path)
                if img is not None:
                    # resize the image
                    resized_img = cv2.resize(img, desired_size)
                    # convert the image to grayscale
                    gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
                    # enhance contrast using histogram equalization
                    enhanced_img = cv2.equalizeHist(gray_img)
                    # apply gamma correction
                    gamma_corrected_img = gamma_correction(enhanced_img, gamma=1.5)
                    # construct the save path for the processed image
                    save_path = os.path.join(save_dir, subdir, filename)
                    os.makedirs(os.path.dirname(save_path), exist_ok=True)  # Create directory if it doesn't exist
                    # save the preprocessed image
                    cv2.imwrite(save_path, gamma_corrected_img)

# **Augmentation**

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment_images(folder_path, num_of_augment):
    # Define the augmentation parameters
    img_gen = ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=False,
        fill_mode='nearest'
    )

    # Loop through all files in the folder path
    for root, _, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith((".jpg", ".jpeg", ".png")):
                img_path = os.path.join(root, filename)

                # Load the original image
                original_img = load_img(img_path, target_size=(250, 250))
                original_img_array = img_to_array(original_img)
                original_img_array = original_img_array.reshape((1,) + original_img_array.shape)

                # Perform augmentation and save the augmented images
                save_to_dir = os.path.dirname(img_path)  # Save in the same subfolder as the original image
                save_prefix = filename.split('.')[0]  # Use the original filename as prefix

                # Generate a batch of augmented images from the single image
                augmented_images = img_gen.flow(original_img_array, batch_size=num_of_augment,
                                                 save_to_dir=save_to_dir, save_prefix=save_prefix,
                                                 save_format='jpeg')

                # Iterate through the generated batch and save each image
                for i in range(num_of_augment):
                    augmented_images.next()

# Call the function with the desired folder path
augment_images('/content/drive/MyDrive/FL23D092/Sign Language Images/Bangla_Sign_Final_datasetV2/augmented_data', 5)

> **Count Images**

In [ ]:
# count the images of each subdirectory
root_dir = '/kaggle/input/augmented-data/Augmented_data'

# specify the image file extensions
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff']

# iterate over all subdirectories in the root directory
for sub_dir in os.listdir(root_dir):
    sub_dir_path = os.path.join(root_dir, sub_dir) # construct the full path to the subdirectory

    # check if the subdirectory exists and is a directory
    if os.path.isdir(sub_dir_path):
        # count the number of image files in the subdirectory
        num_images = sum(1 for f in os.listdir(sub_dir_path) if os.path.splitext(f)[1].lower() in image_extensions)
        # print the subdirectory name and the number of images
        print(f'{sub_dir}: {num_images} images')

In [ ]:
# Count the number of images
dir = '/kaggle/input/augmented-data/Augmented_data'

def count_total(directory):
    num_images = 0
    for f in os.listdir(directory):
        f_path = os.path.join(directory, f)
        # check if the subdirectory or file exists and is a directory
        if os.path.isdir(f_path):
            num_images += count_total(f_path)
        elif os.path.splitext(f)[1].lower() in image_extensions:
            num_images += 1
    return num_images
# function call
num_images = count_total(dir)
print(f'Total number of images: {num_images}')

**Check classes**

In [ ]:
#Load the dataset the JPES files what we converted before
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/augmented-data/Augmented_data',
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

In [ ]:
#Class name in my dataset
class_name=dataset.class_names
class_name

In [ ]:
#image shape
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

**Train Test**

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.7, val_split=0.2, test_split=0.1, shuffle=True, shuffle_size=50):
    assert (train_split + test_split + val_split) == 1

    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds=get_dataset_partitions_tf(dataset)

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
train_size = 0
for _ in train_ds:
    train_size += 1

val_size = 0
for _ in val_ds:
    val_size += 1

test_size = 0
for _ in test_ds:
    test_size += 1

print(f"Training set size: {train_size}")
print(f"Validation set size: {val_size}")
print(f"Test set size: {test_size}")

In [ ]:
BATCH_SIZE = 32  # Adjust batch size as needed
EPOCHS = 40  # Adjust number of epochs as needed
LEARNING_RATE = 0.001  # Adjust learning rate as needed

**Showing random data**

In [ ]:
def visualize_predictions(dataset, class_names, num_images=9):
    prop = fm.FontProperties(fname=font_path)
    plt.figure(figsize=(10, 10))
    for images, labels in dataset.take(1):
        for i in range(num_images):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype('uint8'))
            plt.title(class_names[labels[i].numpy()], fontproperties=prop)
            plt.axis('off')
        plt.show()

visualize_predictions(train_ds, class_name)

# **ResNet50**

In [ ]:
# Load the ResNet50 model with pre-trained weights, excluding the top layer
pretrained_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze all layers in the pre-trained model
for layer in pretrained_model.layers:
    layer.trainable = False

# Add new layers on top of the pre-trained model
x = pretrained_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(46, activation='softmax')(x)  # Output layer with 46 classes

# Create the final model
ResNet50_model = Model(inputs = pretrained_model.input, outputs = predictions)

ResNet50_model.compile(
    optimizer = Adam(learning_rate = LEARNING_RATE),
    loss = SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['accuracy']
)

In [ ]:
history = ResNet50_model.fit(
    train_ds,
    epochs = 50,  # Number of epochs
    validation_data = val_ds,  # Validation data
    steps_per_epoch = len(train_ds),  # Steps per epoch
    validation_steps = len(val_ds)  # Validation steps
)

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = ResNet50_model.evaluate(test_ds)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Save the model
ResNet50_model.save('/kaggle/working/ResNet50_model.h5')

In [ ]:
# Verify the saved model
!ls /kaggle/working

In [ ]:
def visualize_predictions(ResNet50_model, dataset, class_names, num_images=9):
    plt.figure(figsize=(15, 15))
    images_shown = 0
    prop = fm.FontProperties(fname=font_path)

    for images, labels in dataset.take(1):
        predictions = ResNet50_model.predict(images)
        predicted_labels = tf.argmax(predictions, axis=1)
        
        for i in range(num_images):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(f"True: {class_names[labels[i]]}\n Pred: {class_names[predicted_labels[i]]}", fontproperties=prop)
            plt.axis("off")
            print( f"{i}True: {class_names[labels[i]]}\n{i}Pred: {class_names[predicted_labels[i]]}")
            images_shown += 1
            if images_shown >= num_images:
                break
visualize_predictions(ResNet50_model, test_ds, class_name)

In [ ]:
def plot_confusion_matrix(ResNet50_model, dataset, class_names, font_path):
    true_labels = []
    predicted_labels = []

    for images, labels in dataset:
        predictions = ResNet50_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())

    # Calculate the confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    # Load the Bangla font
    prop = fm.FontProperties(fname=font_path)

    # Plot the confusion matrix
    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_names, yticklabels=class_names, cmap="Blues")
    # Set the font properties
    plt.xticks(rotation= 45, ha="right", fontproperties=prop)
    plt.yticks(rotation= 0, fontproperties=prop)
    plt.xlabel('Predicted', fontproperties=prop)
    plt.ylabel('True', fontproperties=prop)
    plt.title('Confusion Matrix', fontproperties=prop)
    plt.show()

# Assuming you have a ResNet50_model, test_ds, and class_name defined
plot_confusion_matrix(ResNet50_model, test_ds, class_name, font_path)

In [ ]:
def print_classification_report(ResNet50_model, dataset, class_names):
    true_labels = []
    predicted_labels = []
    
    for images, labels in dataset:
        predictions = ResNet50_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())
    
    report = classification_report(true_labels, predicted_labels, target_names=class_names)
    print(report)

print_classification_report(ResNet50_model, test_ds, class_name)

# **DenseNet121**

In [ ]:
conv_base = DenseNet201(
    include_top=False,
    weights='imagenet')

for layer in conv_base.layers:
    layer.trainable = False

# Custom classifier layers
x = conv_base.output
#x = GlobalAveragePooling2D()(x)
#x = layers.Dense(1024, activation='relu')(x)  # Experiment with number of neurons


x= GlobalAveragePooling2D()(x)
x= BatchNormalization()(x)
x= Dropout(0.5)(x)
x= Dense(2048,activation='relu')(x) 
x= Dense(1024,activation='relu')(x) 
x= Dense(512,activation='relu')(x) 
x= Dense(256,activation='relu')(x)
x= BatchNormalization()(x)
x= Dropout(0.5)(x)

predictions = layers.Dense(46, activation='softmax')(x)

DenseNet121_model = Model(conv_base.input, predictions)
DenseNet121_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
#Train the model
history = DenseNet121_model.fit(
    train_ds,
    batch_size = BATCH_SIZE,
    validation_data = val_ds,
    verbose = 1,
    epochs = EPOCHS
)

75---densenet121
70---201

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = DenseNet121_model.evaluate(test_ds)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Save the model
DenseNet121_model.save('/kaggle/working/DenseNet121_model.h5')

In [ ]:
# Verify the saved model
!ls /kaggle/working

In [ ]:
def visualize_predictions(DenseNet121_model, dataset, class_names, num_images=9):
    plt.figure(figsize=(15, 15))
    images_shown = 0
    prop = fm.FontProperties(fname=font_path)

    for images, labels in dataset.take(1):
        predictions = DenseNet121_model.predict(images)
        predicted_labels = tf.argmax(predictions, axis=1)
        
        for i in range(num_images):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(f"True: {class_names[labels[i]]}\n Pred: {class_names[predicted_labels[i]]}", fontproperties=prop)
            plt.axis("off")
            print( f"{i}True: {class_names[labels[i]]}\n{i}Pred: {class_names[predicted_labels[i]]}")
            images_shown += 1
            if images_shown >= num_images:
                break
visualize_predictions(DenseNet121_model, test_ds, class_name)

In [ ]:
def plot_confusion_matrix(DenseNet121_model, dataset, class_names, font_path):
    true_labels = []
    predicted_labels = []

    for images, labels in dataset:
        predictions = DenseNet121_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())

    # Calculate the confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    # Load the Bangla font
    prop = fm.FontProperties(fname=font_path)

    # Plot the confusion matrix
    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_names, yticklabels=class_names, cmap="Blues")
    # Set the font properties
    plt.xticks(rotation= 45, ha="right", fontproperties=prop)
    plt.yticks(rotation= 0, fontproperties=prop)
    plt.xlabel('Predicted', fontproperties=prop)
    plt.ylabel('True', fontproperties=prop)
    plt.title('Confusion Matrix', fontproperties=prop)
    plt.show()

# Assuming you have a DenseNet121_model, test_ds, and class_name defined
plot_confusion_matrix(DenseNet121_model, test_ds, class_name, font_path)

In [ ]:
def print_classification_report(DenseNet121_model, dataset, class_names):
    true_labels = []
    predicted_labels = []
    
    for images, labels in dataset:
        predictions = DenseNet121_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())
    
    report = classification_report(true_labels, predicted_labels, target_names=class_names)
    print(report)

print_classification_report(DenseNet121_model, test_ds, class_name)

> **CNN**

In [ ]:
# Define the CNN architecture with input shape (256, 256, 3)
CNN_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),  # Add dropout to prevent overfitting

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),  # Add dropout to prevent overfitting

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),  # Add dropout to prevent overfitting

    Flatten(),
    Dense(256, activation='relu'),  # Increase the number of units in the dense layer
    Dropout(0.2),  # Add dropout to prevent overfitting
    Dense(128, activation='relu'),
    Dropout(0.2),  # Add dropout to prevent overfitting
    Dense(46, activation='softmax')  # Updated to 46 output classes
])

# Compile the model
CNN_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use sparse categorical crossentropy for integer labels
              metrics=['accuracy'])



In [ ]:
# Define the CNN architecture with input shape (256, 256, 3)
CNN_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),  # Add dropout to prevent overfitting

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),  # Add dropout to prevent overfitting

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),  # Add dropout to prevent overfitting

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),  # Add dropout to prevent overfitting

    Conv2D(512, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),  # Add dropout to prevent overfitting

    Conv2D(1024, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),  # Add dropout to prevent overfitting

    Flatten(),
    Dense(256, activation='relu'),  # Increase the number of units in the dense layer
    Dropout(0.2),  # Add dropout to prevent overfitting
    Dense(128, activation='relu'),
    Dropout(0.2),  # Add dropout to prevent overfitting
    Dense(46, activation='softmax')  # Updated to 46 output classes
])

# Compile the model
CNN_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use sparse categorical crossentropy for integer labels
              metrics=['accuracy'])

In [ ]:
# Train the model
history = CNN_model.fit(
    train_ds,
    validation_data=val_ds,
    verbose=1,
    epochs= EPOCHS
)

In [ ]:
test_loss, test_accuracy = CNN_model.evaluate(test_ds)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

> **VGG16**

In [ ]:
# Load the VGG16 model with pre-trained weights, excluding the top layer
pretrained_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze all layers in the pre-trained model
for layer in pretrained_model.layers:
    layer.trainable = False

# Add new layers on top of the pre-trained model
x = pretrained_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)  # Optional: Add dropout for regularization
predictions = layers.Dense(46, activation='softmax')(x)  # Output layer with 46 classes

# Create the final model
VGG16_model = Model(inputs=pretrained_model.input, outputs=predictions)

VGG16_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss=SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
# Train the model
VGG16_history = VGG16_model.fit(
    train_ds,
    epochs=40,  # Adjust the number of epochs as needed
    validation_data=val_ds
)

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = VGG16_model.evaluate(test_ds)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


In [ ]:
# Plot training & validation accuracy values
plt.plot(VGG16_history.history['accuracy'], label='Train Accuracy')
plt.plot(VGG16_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Save the model
VGG16_model.save('/kaggle/working/VGG16_model.h5')

# Verify the saved model
!ls /kaggle/working/

In [ ]:
# Print the model summary to confirm it's loaded correctly
VGG16_model.summary()

In [ ]:
def visualize_predictions(VGG16_model, dataset, class_names, num_images=9):
    plt.figure(figsize=(15, 15))
    images_shown = 0
    prop = fm.FontProperties(fname=font_path)

    for images, labels in dataset.take(1):
        predictions = VGG16_model.predict(images)
        predicted_labels = tf.argmax(predictions, axis=1)
        
        for i in range(num_images):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(f"True: {class_names[labels[i]]}\n Pred: {class_names[predicted_labels[i]]}", fontproperties=prop)
            plt.axis("off")
            print( f"{i}True: {class_names[labels[i]]}\n{i}Pred: {class_names[predicted_labels[i]]}")
            images_shown += 1
            if images_shown >= num_images:
                break
visualize_predictions(VGG16_model, test_ds, class_name)

In [ ]:
def plot_confusion_matrix(VGG16_model, dataset, class_names, font_path):
    true_labels = []
    predicted_labels = []

    for images, labels in dataset:
        predictions = VGG16_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())

    # Calculate the confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    # Load the Bangla font
    prop = fm.FontProperties(fname=font_path)

    # Plot the confusion matrix
    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_names, yticklabels=class_names, cmap="Blues")

    # Set the font properties
    plt.xticks(rotation= 45, ha="right", fontproperties=prop)
    plt.yticks(rotation= 0, fontproperties=prop)
    plt.xlabel('Predicted', fontproperties=prop)
    plt.ylabel('True', fontproperties=prop)
    plt.title('Confusion Matrix', fontproperties=prop)
    plt.show()

# Assuming you have a VGG16_model, test_ds, and class_name defined
plot_confusion_matrix(VGG16_model, test_ds, class_name, font_path)

In [ ]:
def print_classification_report(VGG16_model, dataset, class_names):
    true_labels = []
    predicted_labels = []
    
    for images, labels in dataset:
        predictions = VGG16_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())
    
    report = classification_report(true_labels, predicted_labels, target_names=class_names)
    print(report)

print_classification_report(VGG16_model, test_ds, class_name)

> **VGG19**

In [ ]:
# Load the VGG19 model with pre-trained weights, excluding the top layer
pretrained_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze all layers in the pre-trained model
for layer in pretrained_model.layers:
    layer.trainable = False

# Add new layers on top of the pre-trained model
x = pretrained_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)  # Optional: Add dropout for regularization
predictions = layers.Dense(46, activation='softmax')(x)   # Output layer with 46 classes

# Create the final model
VGG19_model = Model(inputs=pretrained_model.input, outputs=predictions)
VGG19_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss=SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
# Train the model
VGG19_model_history = VGG19_model.fit(
    train_ds,
    epochs=40,  # Adjust the number of epochs as needed
    validation_data=val_ds
)

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = VGG19_model.evaluate(test_ds)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
# Plot training & validation accuracy values
plt.plot(VGG19_model_history.history['accuracy'], label='Train Accuracy')
plt.plot(VGG19_model_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Save the model
VGG19_model.save('/kaggle/working/VGG19_model.h5')

# Verify the saved model
!ls /kaggle/working/

In [ ]:
# Print the model summary to confirm it's loaded correctly
VGG19_model.summary()

In [ ]:
def visualize_predictions(VGG19_model, dataset, class_names, num_images=9):
    plt.figure(figsize=(15, 15))
    images_shown = 0
    prop = fm.FontProperties(fname=font_path)

    for images, labels in dataset.take(1):
        predictions = VGG19_model.predict(images)
        predicted_labels = tf.argmax(predictions, axis=1)
        
        for i in range(num_images):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(f"True: {class_names[labels[i]]}\n Pred: {class_names[predicted_labels[i]]}", fontproperties=prop)
            plt.axis("off")
            print( f"{i}True: {class_names[labels[i]]}\n{i}Pred: {class_names[predicted_labels[i]]}")
            images_shown += 1
            if images_shown >= num_images:
                break
visualize_predictions(VGG19_model, test_ds, class_name)

In [ ]:
def plot_confusion_matrix(VGG19_model, dataset, class_names, font_path):
    true_labels = []
    predicted_labels = []

    for images, labels in dataset:
        predictions = VGG19_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())

    # Calculate the confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    # Load the Bangla font
    prop = fm.FontProperties(fname=font_path)

    # Plot the confusion matrix
    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_names, yticklabels=class_names, cmap="Blues")

    # Set the font properties
    plt.xticks(rotation= 45, ha="right", fontproperties=prop)
    plt.yticks(rotation= 0, fontproperties=prop)
    plt.xlabel('Predicted', fontproperties=prop)
    plt.ylabel('True', fontproperties=prop)
    plt.title('Confusion Matrix', fontproperties=prop)
    plt.show()

# Assuming you have a VGG19_model, test_ds, and class_name defined
plot_confusion_matrix(VGG19_model, test_ds, class_name, font_path)

In [ ]:
def print_classification_report(VGG19_model, dataset, class_names):
    true_labels = []
    predicted_labels = []
    
    for images, labels in dataset:
        predictions = VGG19_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())
    
    report = classification_report(true_labels, predicted_labels, target_names=class_names)
    print(report)

print_classification_report(VGG19_model, test_ds, class_name)

> **InceptionV3**

In [ ]:
# Load the InceptionV3 model with pre-trained weights, excluding the top layer
pretrained_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Unfreeze the last few layers of the pre-trained model
for layer in pretrained_model.layers[-20:]:
    layer.trainable = True
    
# Add new layers on top of the pre-trained model
x = pretrained_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(2048, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)  # Increased number of neurons
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(46, activation='softmax')(x)  # Output layer with 46 classes

# Create the final model
InceptionV3_model = Model(inputs=pretrained_model.input, outputs=predictions)

InceptionV3_model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),  # Adjust the learning rate as needed
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Learning rate scheduler
lr_reduce = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=0.00001
)

# Early stopping to prevent overfitting
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
# Train the model
history = InceptionV3_model.fit(
    train_ds,
    epochs=40,  # Adjust the number of epochs as needed
    validation_data=val_ds,
    callbacks=[lr_reduce, early_stopping]
)

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = InceptionV3_model.evaluate(test_ds)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Save the model
InceptionV3_model.save('/kaggle/working/InceptionV3_model.h5')

# Verify the saved model
!ls /kaggle/working/

In [ ]:
InceptionV3_model.summary()

In [ ]:
def visualize_predictions(InceptionV3_model, dataset, class_names, num_images=9):
    plt.figure(figsize=(15, 15))
    images_shown = 0
    prop = fm.FontProperties(fname=font_path)

    for images, labels in dataset.take(1):
        predictions = InceptionV3_model.predict(images)
        predicted_labels = tf.argmax(predictions, axis=1)
        
        for i in range(num_images):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(f"True: {class_names[labels[i]]}\n Pred: {class_names[predicted_labels[i]]}", fontproperties=prop)
            plt.axis("off")
            print( f"{i}True: {class_names[labels[i]]}\n{i}Pred: {class_names[predicted_labels[i]]}")
            images_shown += 1
            if images_shown >= num_images:
                break
visualize_predictions (InceptionV3_model, test_ds, class_name)

In [ ]:
def plot_confusion_matrix(InceptionV3_model, dataset, class_names, font_path):
    true_labels = []
    predicted_labels = []

    for images, labels in dataset:
        predictions = InceptionV3_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())

    # Calculate the confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    # Load the Bangla font
    prop = fm.FontProperties(fname=font_path)

    # Plot the confusion matrix
    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_names, yticklabels=class_names, cmap="Blues")

    # Set the font properties
    plt.xticks(rotation= 45, ha="right", fontproperties=prop)
    plt.yticks(rotation= 0, fontproperties=prop)
    plt.xlabel('Predicted', fontproperties=prop)
    plt.ylabel('True', fontproperties=prop)
    plt.title('Confusion Matrix', fontproperties=prop)
    plt.show()

# Assuming you have a InceptionV3_model, test_ds, and class_name defined
plot_confusion_matrix(InceptionV3_model, test_ds, class_name, font_path)

In [ ]:
def print_classification_report(InceptionV3_model, dataset, class_names):
    true_labels = []
    predicted_labels = []
    
    for images, labels in dataset:
        predictions = InceptionV3_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())
    
    report = classification_report(true_labels, predicted_labels, target_names=class_names)
    print(report)

print_classification_report(InceptionV3_model, test_ds, class_name)

# **Hybrid**

In [ ]:
num_classes = len(class_name)

# Load InceptionV3 and ResNet50 with pre-trained weights
input_shape = (256, 256, 3)
inception_base = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Fine-tune the top layers of InceptionV3 and ResNet50
for layer in inception_base.layers[-50:]:
    layer.trainable = True
for layer in resnet_base.layers[-50:]:
    layer.trainable = True

# Create input layer
input_tensor = Input(shape=input_shape)

# Extract features using both modelsa
inception_features = inception_base(input_tensor)
resnet_features = resnet_base(input_tensor)

# Pool the features
inception_pool = GlobalAveragePooling2D()(inception_features)
resnet_pool = GlobalAveragePooling2D()(resnet_features)

# Concatenate the pooled features
combined_features = Concatenate()([inception_pool, resnet_pool])

# Add fully connected layers with L2 regularization
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(combined_features)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
output_tensor = Dense(num_classes, activation='softmax')(x)

# Create the hybrid model
hybrid_model = Model(inputs=input_tensor, outputs=output_tensor)

# Compile the model with a lower initial learning rate and a learning rate scheduler
hybrid_model.compile(optimizer=Adam(learning_rate=0.00001),
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

# Learning rate scheduler
lr_reduce = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=0.00001
)

# Early stopping to prevent overfitting
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
# Train the model with augmented data and fine-tuned layers
history = hybrid_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=60,
    callbacks=[early_stopping, lr_reduce]
)

In [ ]:
test_loss, test_accuracy = hybrid_model.evaluate(test_ds)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Plotting the accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plotting the loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Save the model
hybrid_model.save('/kaggle/working/hybrid_model.h5')

# Verify the saved model
!ls /kaggle/working/

In [ ]:
# Print the model summary to confirm it's loaded correctly
hybrid_model.summary()

In [ ]:
def visualize_predictions(hybrid_model, dataset, class_names, num_images=9):
    plt.figure(figsize=(15, 15))
    images_shown = 0
    prop = fm.FontProperties(fname=font_path)

    for images, labels in dataset.take(1):
        predictions = hybrid_model.predict(images)
        predicted_labels = tf.argmax(predictions, axis=1)
        
        for i in range(num_images):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(f"True: {class_names[labels[i]]}\n Pred: {class_names[predicted_labels[i]]}", fontproperties=prop)
            plt.axis("off")
            print( f"{i}True: {class_names[labels[i]]}\n{i}Pred: {class_names[predicted_labels[i]]}")
            images_shown += 1
            if images_shown >= num_images:
                break
visualize_predictions(hybrid_model, test_ds, class_name)

In [ ]:
def plot_confusion_matrix(hybrid_model, dataset, class_names, font_path):
    true_labels = []
    predicted_labels = []

    for images, labels in dataset:
        predictions = hybrid_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())

    # Calculate the confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    # Load the Bangla font
    prop = fm.FontProperties(fname=font_path)

    # Plot the confusion matrix
    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_names, yticklabels=class_names, cmap="Blues")

    # Set the font properties
    plt.xticks(rotation= 45, ha="right", fontproperties=prop)
    plt.yticks(rotation= 0, fontproperties=prop)
    plt.xlabel('Predicted', fontproperties=prop)
    plt.ylabel('True', fontproperties=prop)
    plt.title('Confusion Matrix', fontproperties=prop)
    plt.show()

# Assuming you have a hybrid_model, test_ds, and class_name defined
plot_confusion_matrix(hybrid_model, test_ds, class_name, font_path)

In [ ]:
def print_classification_report(hybrid_model, dataset, class_names):
    true_labels = []
    predicted_labels = []
    
    for images, labels in dataset:
        predictions = hybrid_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())
    
    report = classification_report(true_labels, predicted_labels, target_names=class_names)
    print(report)

print_classification_report(hybrid_model, test_ds, class_name)

> **Xecpetion Model**

In [ ]:
# Define number of classes
num_classes = len(class_name)

# Load Xception with pre-trained weights
input_shape = (256, 256, 3)
xception_base = Xception(weights='imagenet', include_top=False, input_shape=input_shape)

# Fine-tune the top layers of Xception
for layer in xception_base.layers[-50:]:
    layer.trainable = True

# Create input layer
input_tensor = Input(shape=input_shape)

# Extract features using Xception
xception_features = xception_base(input_tensor)

# Pool the features
xception_pool = GlobalAveragePooling2D()(xception_features)

# Add fully connected layers with L2 regularization
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(xception_pool)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
output_tensor = Dense(num_classes, activation='softmax')(x)
# Create the Xception model
xception_model = Model(inputs=input_tensor, outputs=output_tensor)

# Compile the model with a lower initial learning rate and a learning rate scheduler
xception_model.compile(optimizer=Adam(learning_rate=0.00001),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])
# Learning rate scheduler
lr_reduce = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=0.00001
)

# Early stopping to prevent overfitting
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
# Train the model with more epochs and fine-tuned layers
history = xception_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs = EPOCHS,  # Increased the number of epochs
    callbacks=[early_stopping, lr_reduce]
)

In [ ]:
# Evaluate the Xception model on the test set
test_loss, test_accuracy = xception_model.evaluate(test_ds)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Save the model
hybrid_model.save('/kaggle/working/xception_model.h5')

# Verify the saved model
!ls /kaggle/working/

In [ ]:
# Print the model summary to confirm it's loaded correctly
xception_model.summary()

In [ ]:
def visualize_predictions(xception_model, dataset, class_names, num_images=9):
    plt.figure(figsize=(15, 15))
    images_shown = 0
    prop = fm.FontProperties(fname=font_path)

    for images, labels in dataset.take(1):
        predictions = xception_model.predict(images)
        predicted_labels = tf.argmax(predictions, axis=1)
        
        for i in range(num_images):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(f"True: {class_names[labels[i]]}\n Pred: {class_names[predicted_labels[i]]}", fontproperties=prop)
            plt.axis("off")
            print( f"{i}True: {class_names[labels[i]]}\n{i}Pred: {class_names[predicted_labels[i]]}")
            images_shown += 1
            if images_shown >= num_images:
                break
visualize_predictions(xception_model, test_ds, class_name)

In [ ]:
def plot_confusion_matrix(xception_model, dataset, class_names, font_path):
    true_labels = []
    predicted_labels = []

    for images, labels in dataset:
        predictions = xception_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())

    # Calculate the confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    # Load the Bangla font
    prop = fm.FontProperties(fname=font_path)

    # Plot the confusion matrix
    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_names, yticklabels=class_names, cmap="Blues")

    # Set the font properties
    plt.xticks(rotation= 45, ha="right", fontproperties=prop)
    plt.yticks(rotation= 0, fontproperties=prop)
    plt.xlabel('Predicted', fontproperties=prop)
    plt.ylabel('True', fontproperties=prop)
    plt.title('Confusion Matrix', fontproperties=prop)
    plt.show()

# Assuming you have a xception_model, test_ds, and class_name defined
plot_confusion_matrix(xception_model, test_ds, class_name, font_path)

In [ ]:
def print_classification_report(xception_model, dataset, class_names):
    true_labels = []
    predicted_labels = []
    
    for images, labels in dataset:
        predictions = xception_model.predict(images)
        predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        true_labels.extend(labels.numpy())
    
    report = classification_report(true_labels, predicted_labels, target_names=class_names)
    print(report)

print_classification_report(xception_model, test_ds, class_name)

> **MobileNetV2**

In [ ]:
# Import necessary libraries
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load the MobileNetV2 model with pre-trained weights, excluding the top layer
pretrained_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze all layers in the pre-trained model
for layer in pretrained_model.layers:
    layer.trainable = False

# Add new layers on top of the pre-trained model
x = pretrained_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)  # Add dropout layer
x = BatchNormalization()(x)
predictions = Dense(46, activation='softmax')(x)  # Output layer with 46 classes

# Create the final model
MobileNetV2_model = Model(inputs=pretrained_model.input, outputs=predictions)

# Compile the model with a different optimizer and learning rate
MobileNetV2_model.compile(
    optimizer=SGD(learning_rate=0.001),
    loss=SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
# Train the model with more epochs and fine-tuned layers
history = MobileNetV2_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs = 40,  # Increased the number of epochs
    callbacks=[early_stopping, lr_reduce]
)

In [ ]:
# Evaluate the MobileNetV2 model on the test set
test_loss, test_accuracy = MobileNetV2_model.evaluate(test_ds)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

In [ ]:

# Set input shape
input_shape = (256, 256, 3)
num_classes = 46
# Load pre-trained models with pre-trained weights
inception_base = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the convolutional layers of both models (adjust as needed)
for layer in inception_base.layers[:-100]:
  layer.trainable = False
for layer in vgg16_base.layers[:-100]:
  layer.trainable = False

# Create the model input
input_tensor = Input(shape=input_shape)

# Extract features from both models
inception_features = inception_base(input_tensor)
vgg16_features = vgg16_base(input_tensor)

# Pool the features
inception_pool = GlobalAveragePooling2D()(inception_features)
vgg16_pool = GlobalAveragePooling2D()(vgg16_features)

# Concatenate the features
combined_features = Concatenate()([inception_pool, vgg16_pool])

# Add fully-connected layers with L2 regularization and dropout
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(combined_features)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
output_tensor = Dense(num_classes, activation='softmax')(x)

# Create the final hybrid model
hybrid_model = Model(inputs=input_tensor, outputs=output_tensor)

# Compile the model with Adam optimizer and categorical crossentropy loss
hybrid_model.compile(optimizer=Adam(learning_rate=0.00001),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

# Learning rate scheduler
lr_reduce = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=0.00001
)

# Early stopping to prevent overfitting
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)


In [ ]:
# Train the model with more epochs and fine-tuned layers
history = hybrid_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs = 40,  # Increased the number of epochs
    callbacks=[early_stopping, lr_reduce]
)